In [1]:
from getdist import mcsamples, plots, chains
from getdist.mcsamples import MCSamplesError
import numpy as np
from subprocess import run
import os
from yaml import dump
from cobaya.yaml import yaml_load_file
from copy import deepcopy
from time import time

In [40]:
class lkl_prof:
    
    def __init__(self, chains_dir, chain_file, prof_param, processes=6, R_minus_1_wanted=0.05, 
                 mcmc_chain_settings={'ignore_rows' : 0.3}, 
                 minimizer_settings={'minimize': {'method': 'bobyqa','covmat' : 'auto',}}, 
                 prof_incr=None, prof_min=None, prof_max=None
                ):
        self.chains_dir = chains_dir
        self.chain_file = chain_file
        
        self.processes = processes
        self.R_minus_1_wanted = R_minus_1_wanted
        self.mcmc_chain_settings = mcmc_chain_settings
        self.minimizer_settings = minimizer_settings
        
        self.prof_param = prof_param
        self.prof_incr = prof_incr
        self.prof_min = prof_min
        self.prof_max = prof_max
        
        os.chdir(self.chains_dir)
    
    def check_mcmc_chains(self):
        """
        Check if mcmc chains chains exist 
        
        :return: True if files found, else False 
        """
        try:
            self.mcmc_chains = mcsamples.loadMCSamples(self.chains_dir+self.chain_file, settings=self.mcmc_chain_settings)
            return True
        except OSError:
            return False 
        
    def run_mcmc(self, resume=False):
        """
        Run or resume MCMC chains 
        
        :return: True if files found, else False 
        """
        self.mcmc_yaml = yaml_load_file(self.chains_dir+self.chain_file+'.yaml')
        try:
            self.mcmc_yaml['sampler']['mcmc']['Rminus1_stop'] = self.R_minus_1_wanted
        except KeyError:
            print("Error: Input yaml not set up correctly for an mcmc run. Please ensure the sampler 'mcmc' is correctly set. ")
            raise KeyError
        if self.mcmc_yaml['output'].count('/') > 1:
            print("Error: For correct functioning of the code, please have the input .yaml file in the same directory as the output chains. Currently, output is: "+self.mcmc_yaml['output'])
            raise OSError
        if resume==False:
            run("mpirun -np "+str(self.processes)+" cobaya-run "+self.chain_file+".yaml", shell=True)
        else:
            run("mpirun -np "+str(self.processes)+" cobaya-run "+self.chain_file+".yaml -r", shell=True)
        
        return True
        
    def check_mcmc_convergence(self, mcmc_chains=None):
        """
        Check if MCMC converged 
        
        :mcmc_chains: getdist MCSamples instance 
        
        :return: True if MCMC chains have converged to the desired R-1, default is R-1=0.05. Else False 
        """
        if mcmc_chains==None:
            mcmc_chains=self.mcmc_chains
            
        current_R_minus_1 = mcmc_chains.getGelmanRubin()
        if current_R_minus_1 < self.R_minus_1_wanted:
            print("Chains converged sufficiently. Current R-1 = {:.3f} satisfies R-1 wanted = {:.3f}. \nMove on to checking minimum.".format(current_R_minus_1,self.R_minus_1_wanted))
            return True
        else: 
            print("Chains not converged. Current R-1 = {:.3f} while R-1 wanted = {:.3f}. \nResuming MCMC. ".format(current_R_minus_1,self.R_minus_1_wanted))
            return False 
    
    def mcmc(self):
        """
        Check MCMC and run if needed 
        
        :return: True once finished 
        """
        if not self.check_mcmc_chains():
            self.run_mcmc()
        elif not self.check_mcmc_convergence():
            self.run_mcmc(resume=True)
        return True

    def check_global_min(self, mcmc_chains=None):
        """
        Check if minimizer was run
        
        :mcmc_chains: getdist MCSamples instance 
        
        :return: True if global minimum was run and relevant files are accesible. Else False 
        """
        if mcmc_chains==None:
            mcmc_chains=self.mcmc_chains
            
        try:
            mcmc_chains.getParamBestFitDict()
            min_yaml = yaml_load_file(self.chains_dir+self.chain_file+'.minimize.updated.yaml')
            print("check_global_min: Found previously run MCMC chains and global minimizer. ")
            return True
        except MCSamplesError:
            print("check_global_min: Need to first run a minimizer on the full MCMC chains before beginning 1d profile lkl code.")
            return False 
        except FileNotFoundError:
            print("check_global_min: Found best-fit but not the file "+self.chains_dir+self.chain_file+".minimize.updated.yaml. Something has gone wrong. ")
            return FileNotFoundError    
        
    def global_min(self):
        """
        Check global minizer, run if needed, then write if not already written 
        
        So: 
        1) load the global minimum file. 
        2) check if we have a file with prof lkl values. 
            * If yes, check that it has the same parameters and in the right order. Proceed. 
            * If no file, start it and write the first line as param names. Proceed. 
            * If file yes, but parameters don't match, then print an error. Stop. 
        2) check if global minimum params have already been written (first line of file)
            * If parameters are written, check that they match global minimum. Don't write them again
            * If parameters are written but don't match, spit out error. 
            * If no params written, add this current ML values for all parameters in append mode
        
        :return: global maximum lkl dictionary 
        """
        if not self.check_global_min():
            self.min_yaml = yaml_load_file(self.chains_dir+self.chain_file+'.input.yaml')
            self.min_yaml['sampler'] = self.minimizer_settings

            with open(self.chains_dir + self.chain_file + '.minimize.input.yaml', 'w') as yaml_file:
                dump(self.min_yaml, yaml_file, default_flow_style=False)

            self.run_minimizer(yaml_ext='') # had also passed debug=True here. Can't remember why.  
            
        param_names, param_ML, MLs = self.read_minimum(extension='')
        self.global_ML = deepcopy(MLs)
        self.param_order = param_names
        
        try:
            self.match_param_names(self.param_order)
        except FileNotFoundError:
            extension = '_lkl_profile.txt'
            print("File not found. Starting a new file now: " + self.chains_dir + self.chain_file + self.pn_ext(extension) + '\n')
            with open(self.chains_dir + self.chain_file + self.pn_ext(extension), 'w') as lkl_txt:
                lkl_txt.write("#")
                for param_recorded in self.param_order:
                    lkl_txt.write("\t %s" % param_recorded)
                lkl_txt.write("\n")
                
        extension = '_lkl_profile.txt'
        lkl_prof_table = np.loadtxt(self.chains_dir + self.chain_file + self.pn_ext(extension))

        if lkl_prof_table.shape!=(0,):
            if not self.match_param_line(self.global_ML, loc=0):
                print("Something went wrong. The first line of the lkl_profile.txt file which should be global ML does not match the global ML in file \n"
                     +self.chains_dir + self.chain_file + '.minimum')
                raise FileExistsError
        else: 
            self.write_MLs()
            
        return self.global_ML
        
        
    def pn_ext(self, extension):
        """
        Prefix the file extension string input with the sign of the profile lkl parameter increment to track files correctly. 
        
        :extension: A string of the file name extension without the sign of the prof_parameter 
        :return: String of extension prefixed with the sign of the increment of profile lkl parameter 
        """
        if len(extension)>0:
            if self.prof_incr > 0:
                extension = '_p'+extension
            if self.prof_incr < 0:
                extension = '_n'+extension
        return extension
        
    def read_minimum(self, extension='_lkl_prof'):
        """
        Read minimum file and save parameter names list, parameter values list and MLs dictionary 
        
        :extension: The extension of the life type being read in. Leave this as is, the rest of the code assumes the same naming conventions. 
        
        :return: List of parameter names, list of parameter ML values, dictionary of {'param_names': param_ML_value}
        """
        extension=self.pn_ext(extension)
        
        param_ML, param_names = np.loadtxt(self.chains_dir + self.chain_file + extension + '.minimum', skiprows=3, usecols = (1,2), dtype=str, unpack=True)
        param_ML = param_ML.astype(float)
        
        with open(self.chains_dir + self.chain_file + extension + '.minimum') as min_file:
            loglkl_and_chi = [next(min_file) for x in range(2)] # reading in the first two lines separately for -log(lkl) and chi^2
        for line in loglkl_and_chi:
            param_names = np.append(param_names, line.split("=")[0])
            param_ML = np.append(param_ML, float(line.split("=")[1]))
        MLs = dict(zip(param_names, param_ML))
        
        self.param_names = param_names
        self.MLs = MLs
        
        return param_names, param_ML, MLs
    
    def read_lkl_output(self, extension='_lkl_profile.txt', loc=-1):
        """
        Read (default = last) line of lkl prof output file into list
        
        :extension: Leave this alone, thank you. 
        :loc: integer location of line in file to read. Default is last line 
        
        :return: List of parameters
        """

        extension=self.pn_ext(extension)

        lkl_prof_table = np.loadtxt(self.chains_dir + self.chain_file + extension)
        try:
            lkl_prof_table.shape[1] # check that lkl_prof_table has multiple rows
            lkl_prof_table = lkl_prof_table[loc, :]
        except IndexError:
            pass
        return lkl_prof_table
    
    def write_MLs(self, MLs=None, extension='_lkl_profile.txt'):
        """
        Write params from MLs dict into txt file in append mode
        Note that to write, we use self.param_order, not self.param_names. 
        This is because the global param_names list is the one that has the correct order. 
        
        :extension: Leave it alone, thank you.
        
        :return: new length of the saved lkl profile table
        
        """
        if MLs == None:
            MLs = self.MLs
        extension=self.pn_ext(extension)
        
        with open(self.chains_dir + self.chain_file + extension, 'a') as lkl_txt:
            for param in self.param_order:
                lkl_txt.write("\t %s" % str(MLs[param]))
            lkl_txt.write("\n")
        lkl_prof_table = np.loadtxt(self.chains_dir + self.chain_file + extension)
        return lkl_prof_table.shape
    
    
    def match_param_names(self, param_names, extension='_lkl_profile.txt'):
        """
        Check that param names match in target file and MLs dictionary
        
        :param_names: List of param_names to check against the file 
        :extension: Leave it alone, thank you. 
        
        :return: True if param_names match, else False
        """
        
        extension=self.pn_ext(extension)
        
        with open(self.chains_dir + self.chain_file + extension, 'r') as lkl_txt:
            params_recorded = lkl_txt.readline()
        # define the expected first row of this file
        expected_string = '#'
        for param in param_names:
            expected_string += "\t %s" % param
        expected_string += "\n"
        if expected_string == params_recorded:
            print("match_param_names: Found existing file with correct name and parameters / parameter sequence. Will append to it. \n" 
                     + self.chains_dir + self.chain_file + extension)
            return True
        else:
            print("match_param_names: Error: existing file found at " + self.chains_dir + self.chain_file + extension 
                 + "\n but parameters / parameter sequence does not match expected.")
            print("--> parameters found: \n" + params_recorded)
            print("--> parameters expected: \n" + expected_string)
            raise FileExistsError
            return False
    
    def match_param_line(self, MLs, param_names=None, extension='_lkl_profile.txt', loc=-1):
        """
        Check if specified (default: last) location in lkl_prof output file matches current MLs
        
        :param_names: list of parameter names in the same order as that printed in the file. 
                        This is usually the global param_order list. 
        :MLs: dictionary of {'param_name': ML_value }
        :extension: Leave it alone, thank you. 
        :loc: integer location of row in file to check, default is the last line
        
        :return: True if match, else False 
        """
        
        extension=self.pn_ext(extension)
        
        if param_names==None:
            param_names=self.param_order

        lkl_prof_table = np.loadtxt(self.chains_dir + self.chain_file + extension)
        if lkl_prof_table.size==0:
            print("match_param_line: File empty ")
            return False
        else: 
            try:
                lkl_prof_table.shape[1] # check that lkl_prof_table has multiple rows
                if False in [lkl_prof_table[loc, np.where(param_names == param)] == MLs[param] for param in param_names]:
                    return False
                else:
                    return True 
            except IndexError:
                print("match_param_line: Only one entry in file, checking that entry ")
                if False in [lkl_prof_table[np.where(param_names == param)] == MLs[param] for param in param_names]:
                    return False 
                else:
                    return True               


    def increment_update_yaml(self, MLs, lkl_pro_yaml, yaml_ext = '_lkl_prof'):
        """
        Update yaml info to next increment 
        
        
        :MLs: dictionary of {'param_name': ML_value }
        :lkl_pro_yaml: dictionary of yaml info for running lkl profile incremental minimizer 
        :yaml_ext: Leave it alone. 
        
        :return: dictionary for yaml info of profile lkl parameter, including incremented value and latex info
        """
        yaml_ext=self.pn_ext(yaml_ext)
        
        # update profile lkl param 
        latex_info = lkl_pro_yaml['params'][self.prof_param]['latex']
        lkl_pro_yaml['params'][self.prof_param] = {'value': MLs[self.prof_param]+self.prof_incr, 'latex': latex_info}
        lkl_pro_yaml['output'] = self.chain_file + yaml_ext
        # update all other independent parameters 
        for param in lkl_pro_yaml['params']:
            if 'prior' in lkl_pro_yaml['params'][param]:
                lkl_pro_yaml['params'][param]['ref'] = MLs[param]
        # dump yaml to file for running 
        with open(self.chains_dir+self.chain_file+yaml_ext+'.minimize.input.yaml', 'w') as yaml_file:
            dump(lkl_pro_yaml, yaml_file, default_flow_style=False)    
        return lkl_pro_yaml['params'][self.prof_param]
    
    def run_minimizer(self, yaml_ext='_lkl_prof', debug=False):
        """
        Run minimizer 
        For the parameter we want to vary, remove all but latex and value. 
        The latex is as before from the MCMC yaml file. 
        The value is ML $\pm$ increment. 
        
        :yaml_ext: Leave it alone. 
        :debug: Do you want Cobaya debug output (it's a LOT)
        
        :return: True
        """
        yaml_ext=self.pn_ext(yaml_ext)

        if debug==True:
            run("mpirun -np "+str(self.processes)+" cobaya-run "+self.chain_file+yaml_ext+".minimize.input.yaml -f -d", shell=True)
        else:
            run("mpirun -np "+str(self.processes)+" cobaya-run "+self.chain_file+yaml_ext+".minimize.input.yaml -f", shell=True)   
        return True
    
    def init_lkl_prof(self):
        """
        Initialise profile lkl yaml:
        1) copy the global minimiser yaml into a profile lkl yaml if not already done, 
        and set the sampler to minimiser settings specified in the class call, 
        and the covamt to the global mcmc covmat. 
        2) read the last line of the lkl output file and set that as the current MLs dictionary, as self.MLs. 
        this updates the location in prof_param that we're at for running prof lkls 
        3) set self.lkl_pro_yaml as a dictionary of this modified global minimiser yaml 
        
        :return: the lkl profile yaml dictionary 
        """
        extension = '_lkl_prof'
        try:
            lkl_pro_yaml = yaml_load_file(self.chains_dir+self.chain_file+self.pn_ext(extension)+'.minimize.input.yaml')
        except FileNotFoundError:
            run("cp "+self.chains_dir+self.chain_file+'.minimize.updated.yaml'+" "
                    +self.chains_dir+self.chain_file+self.pn_ext(extension)+'.minimize.input.yaml', shell=True)
            lkl_pro_yaml = yaml_load_file(self.chains_dir+self.chain_file+self.pn_ext(extension)+'.minimize.input.yaml')
        lkl_pro_yaml['sampler'] = self.minimizer_settings
        lkl_pro_yaml['sampler']['minimize']['covmat'] = self.chains_dir+self.chain_file+'.covmat'

        param_ML = self.read_lkl_output()
        self.MLs = dict(zip(self.param_names, param_ML))
        
        self.lkl_pro_yaml = deepcopy(lkl_pro_yaml)
        
        return self.lkl_pro_yaml
        
    def run_lkl_prof(self, time_mins=False):
        """
        Run the likelihood profile loop. 
        Initialise time-keeping file if wanted. 
        
        While we are within the bounds of the profile param we want to explore: 
        1) check if the point we are currently at i.e. param_ML and MLs, matches the last entry in the lkl_prof table.
            - if it does, the last minimum was run and saved successfully.
            - if not, check if a minimum file exists. 
                - if it does, read it in and save it in the lkl prof txt. minimum run successfully. 
                - if not, this happens when we have updated the yaml but the minimizer didn't finish. 
                  Run the yaml again without updating. 
        2) check if minimum was run and saved. 
            - if yes, update the yaml and increment the prof lkl param, 
              update all other params to new values from current ML. 
              Assign the MLs values for the independent params in the yaml as new reference starting points. 
        3) run the minimizer 
        4) save minimizer output 

        :time_mins: boolean for whether you want to time each minimiser increment or not 
        
        :return: the value of the profile lkl parameter at the end of this loop 
        """
        if time_mins == True:
            time_extension = '_time_stamps.txt'
            time_extension = self.pn_ext(time_extension)
            with open(self.chains_dir + self.chain_file + time_extension, 'w') as lkl_txt:
                lkl_txt.write("#")
                lkl_txt.write(" %s \t step_size \t minimizer_time " % self.prof_param)
                lkl_txt.write("\n")

        extension = '_lkl_prof'
        extension = self.pn_ext(extension)

        MLs = deepcopy(self.MLs)
        lkl_pro_yaml = deepcopy(self.lkl_pro_yaml)

        while ((MLs[self.prof_param] < self.prof_max) and (MLs[self.prof_param] > self.prof_min)):
            last_entry_matches_current_params = self.match_param_line(MLs)
            if last_entry_matches_current_params:
                run('rm '+self.chains_dir + self.chain_file + extension + '.minimum*', shell=True)
                minimum_successfully_run_and_saved = True
            else:
                try:
                    param_names, param_ML, MLs = self.read_minimum()
                    self.write_MLs(MLs)
                    run('rm '+self.chains_dir + self.chain_file + extension + '.minimum*', shell=True)
                    minimum_successfully_run_and_saved = True 
                    print("-----> Minimizer run successfully for "+self.prof_param+" = "+str(MLs[self.prof_param]))
                except OSError:
                    minimum_successfully_run_and_saved = False
                    print("-----> Minimizer not run for "+self.prof_param+" = "+str(MLs[self.prof_param]))
                    print("       Rerunning this point")

            if minimum_successfully_run_and_saved:
                self.increment_update_yaml(MLs, lkl_pro_yaml)
                run('rm '+self.chains_dir + self.chain_file + extension + '.minimize.updated.yaml', shell=True)

            time_start = time()

            self.run_minimizer()

            time_end = time()
            time_taken = time_end - time_start

            if time_mins == True:
                with open(self.chains_dir + self.chain_file + time_extension, 'a') as lkl_txt:
                    lkl_txt.write("{:.4g} \t {:.2g} \t {:.2f} \n".format(lkl_pro_yaml['params'][self.prof_param]['value'], 
                                                                         self.prof_incr, time_taken))
                print("       Time taken for minimizer = {:.2f}".format(time_taken))

            param_names, param_ML, MLs = self.read_minimum()

    #         prof_incr *= 2. # Readjust prof lkl increment if wanted by copying this function and adding such a line 

        # outside loop now 
        last_entry_matches_current_params = self.match_param_line(MLs)
        if not last_entry_matches_current_params:
            param_names, param_ML, MLs = self.read_minimum()
            self.write_MLs(MLs)
            print("-----> Minimizer run successfully for "+self.prof_param+" = "+str(MLs[self.prof_param]))
        
        return MLs[self.prof_param]
    
    
    def full_lkl_prof_array(self):
        """
        Combine positive and negative increment files into one array 
        
        :return: full likelihood profile array 
        """
        try:
            all_MLs_p = np.loadtxt(self.chains_dir+self.chain_file+'_p_lkl_profile.txt')
            pos_file = True
        except OSError:
            pos_file = False
        try:
            all_MLs_n = np.loadtxt(self.chains_dir+self.chain_file+'_n_lkl_profile.txt')
            if pos_file==True:
                all_MLs = np.concatenate( (np.flip(all_MLs_n, 0),all_MLs_p) )
            else:
                all_MLs = np.flip(all_MLs_n, 0)
        except OSError:
            all_MLs = all_MLs_p

        return all_MLs

    def full_lkl_prof_dict(self):
        """
        Combine positive and negative increment files into one dictionary with 
        keys = param names 
        values = 1D array of profile likelihood values 
        
        :return: full likelihood profile dictionary 
        """
        full_prof_dict = {}
        full_lkl_prof_array = self.full_lkl_prof_array()

        for param_num in range(len(self.param_order)):
            full_prof_dict[self.param_order[param_num]] = full_lkl_prof_array[:,param_num]

        return full_prof_dict

    def sum_params(self, params_to_sum):
        """
        Sum list of params and return array of summed params.
        Useful for adding up chi^2's post profile lkl run 
        
        :params_to_sum: list of parameter names that you want to sum. 
        
        :return: array of summed parameters 
        """
        prof_lkl = self.full_lkl_prof_dict()

        param_vectors = [prof_lkl[param] for param in params_to_sum]
        param_stack = np.stack(param_vectors, axis=0)
        summed_params = param_stack.sum(axis=0)

        return summed_params
            